<a href="https://colab.research.google.com/github/jokefun022/Google-Drive/blob/main/24_02_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

import re
import string
# !pip install wordcloud
from wordcloud import WordCloud

from textblob import TextBlob

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import word2vec

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.pipeline import Pipeline

# Corrected TensorFlow imports
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import transformers
from transformers import (AutoTokenizer,
                          AutoModelForSequenceClassification,
                          TextClassificationPipeline)

import joblib

# Ensure SHAP is installed
import shap


In [ ]:
data = pd.read_csv("aggression_parsed_dataset.csv")

In [ ]:
data

,index,Text,ed_label_0,ed_label_1,oh_label
0,0,`- This is not ``creative``. Those are the di...,0.900000,0.100000,0
1,1,` :: the term ``standard model`` is itself le...,1.000000,0.000000,0
2,2,"True or false, the situation as of March 200...",1.000000,0.000000,0
3,3,"Next, maybe you could work on being less cond...",0.555556,0.444444,0
4,4,This page will need disambiguation.,1.000000,0.000000,0
...,...,...,...,...,...
115859,115859,` These sources don't exactly exude a sense ...,0.777778,0.222222,0
115860,115860,The Institute for Historical Review is a pee...,1.000000,0.000000,0
115861,115861,:The way you're trying to describe it in this...,1.000000,0.000000,0
115862,115862,== Warning == There is clearly a protection...,0.500000,0.500000,0


In [ ]:
data.isna().sum()

,0
index,0
Text,0
ed_label_0,0
ed_label_1,0
oh_label,0


In [ ]:
# regex:
# - Match patterns in a string
# - Handle when a pattern is found

# Example: re.sub(pattern, replacement, text)

# If an alphabet => keep

# nltk: Natural Language Toolkit
# - Provides dictionaries and collections of words
# - Functions include stemming, lemmatization, and detecting POS-tags

# string module:
# - Contains string functions
# - Example: string.punctuation


In [ ]:
import nltk
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [ ]:
import re
import string
import nltk

# Download required NLTK dataset
nltk.download('words')

only_english = set(nltk.corpus.words.words())

def change_case_lower(text):
    return " ".join([x.lower() for x in text.split()])

def remove_url(text):
    return re.sub(r"\S*https?:\S*", '', text)  # Remove links and URLs

def remove_html(text):
    return re.sub(r'<.*?>', '', text)  # Remove HTML tags

def remove_punctuations(text):
    text = re.sub(r'\[.*?\]', '', text)  # Remove text in [square brackets]
    text = re.sub(r'\(.*?\)', '', text)  # Remove text in (parentheses)
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)  # Remove punctuation
    text = re.sub(r"[''""...“”‘’…]", '', text)  # Remove specific quotation marks
    return text

def remove_special_characters(text):
    text = re.sub(r'\n', ' ', text)  # Remove new line characters
    text = re.sub(r'\\n', ' ', text)  # Remove escaped new lines
    text = ' '.join([w for w in nltk.wordpunct_tokenize(text) if w.lower() in only_english or not w.isalpha()])  # Keep English words
    text = ' '.join([word for word in text.split() if re.search("[a-zA-Z\s]+", word)])  # Remove non-English words
    return text.strip()

def remove_hashtags_total(text):
    return re.sub(r'#', ' ', text)  # Remove all hashtag symbols

def remove_hashtags_only(text):
    return ' '.join([x for x in text.split() if not x.startswith('#')])  # Remove words that start with #

def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticons
                               u"\U0001F300-\U0001F5FF"  # Symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # Transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # Flags
                               u"\U00002500-\U00002BEF"  # Chinese characters
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001F926-\U0001F937"
                               u"\U00010000-\U0010FFFF"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200D"
                               u"\u23CF"
                               u"\u23E9"
                               u"\u231A"
                               u"\uFE0F"  # Dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)  # Remove emojis and symbols


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
only_english = set(nltk.corpus.words.words())
def clean_text(text):

    sample = text
    sample = " ".join([x.lower() for x in sample.split()])
    sample = re.sub(r"\S*https?:\S*", '', sample) #links and urls
    sample = re.sub('\[.*?\]', '', sample) #text between [square brackets]
    sample = re.sub('\(.*?\)', '', sample) #text between (parenthesis)
    sample = re.sub('[%s]' % re.escape(string.punctuation), '', sample) #punctuations
    sample = re.sub('\w*\d\w', '', sample) #digits with trailing or preceeding text
    sample = re.sub(r'\n', ' ', sample) #new line character
    sample = re.sub(r'\\n', ' ', sample) #new line character
    sample = re.sub("[''""...“”‘’…]", '', sample) #list of quotation marks
    sample = re.sub(r', /<[^>]+>/', '', sample)    #HTML attributes

    sample = ' '.join([w for w in nltk.wordpunct_tokenize(sample) if w.lower() in only_english or not w.isalpha()]) #doesn't remove indian languages
    sample = ' '.join(list(filter(lambda ele: re.search("[a-zA-Z\s]+", ele) is not None, sample.split()))) #languages other than english

    sample = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE).sub(r'', sample) #emojis and symbols
    sample = sample.strip()
    sample = " ".join([x.strip() for x in sample.split()])

    return sample


In [ ]:
import pandas as pd
import re
import string
import nltk

# Download NLTK words dataset if not already downloaded
nltk.download('words')

# Load sample data (Replace this with your actual dataset)
# Example dataset with 'text' column
data = pd.DataFrame({'text': ["Hello! Check this out: https://example.com 😊",
                              "<p>This is <b>bold</b> text!</p> #hashtag",
                              "Python is amazing!! #coding #NLP"]})

# Function to clean text
def clean_text(text):
    if not isinstance(text, str):  # Ensure input is a string
        return ""

    only_english = set(nltk.corpus.words.words())

    text = text.lower()  # Convert to lowercase
    text = re.sub(r"\S*https?:\S*", '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(f"[{re.escape(string.punctuation)}]", '', text)  # Remove punctuation
    text = re.sub(r'#', ' ', text)  # Remove hashtag symbols
    text = ' '.join([w for w in nltk.wordpunct_tokenize(text) if w.lower() in only_english or not w.isalpha()])  # Keep only English words
    text = text.strip()  # Remove leading/trailing spaces

    return text

# Check if 'text' column exists before applying function
if 'text' in data.columns:
    data['cleaned_text'] = data['text'].apply(clean_text)
else:
    print("Error: 'text' column not found in the dataset!")

# Display cleaned data
print(data)


[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


                                           text            cleaned_text
0  Hello! Check this out: https://example.com 😊  hello check this out 😊
1     <p>This is <b>bold</b> text!</p> #hashtag       this is bold text
2              Python is amazing!! #coding #NLP       python is amazing


In [ ]:
data['cleaned_text'] = data['text'].apply(lambda x: clean_text())
data

,text,cleaned_text
0,Hello! Check this out: https://example.com 😊,hello check this out 😊
1,<p>This is <b>bold</b> text!</p> #hashtag,this is bold text
2,Python is amazing!! #coding #NLP,python is amazing


In [ ]:
plt.figure(figsize=(15,10))
data['word_count'].value_counts().plot(kind='bar', color=sns.color_palette('pastel'))

KeyError: 'word_count'

<Figure size 1500x1000 with 0 Axes>